In [ ]:
# --- Cell 1: Imports & Utilities (replace existing utility cell) ---
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Config (edit here)
NETCDF_PATH = "../cloud_results.nc"
OUT_DIR = "figs_notebook"
MIN_POINTS = 30          # min max size (grid points)
MIN_LIFETIME = 2         # min active timesteps
USE_LOG_SIZE = False

def open_ds(path): return xr.open_dataset(path)

def valid_track_mask(ds): return (ds['valid_track'] == 1)

def active_mask(ds): return ~np.isnan(ds['size'])

def per_track_lifetime(ds): return active_mask(ds).sum(dim='time')

def filter_tracks(ds, min_points=MIN_POINTS, min_life=MIN_LIFETIME):
    size_max = ds['size'].max(dim='time', skipna=True)
    life = per_track_lifetime(ds)
    return valid_track_mask(ds) & (size_max >= min_points) & (life >= min_life)

def _time_index_first(arr):
    v = arr.values
    idx = np.where(~np.isnan(v))[0]
    return idx[0] if idx.size else np.nan

def _time_index_last(arr):
    v = arr.values
    idx = np.where(~np.isnan(v))[0]
    return idx[-1] if idx.size else np.nan

def build_track_dataframe(ds, track_sel=None):
    if track_sel is None: track_sel = filter_tracks(ds)
    ds_sel = ds.sel(track=track_sel)

    size_max = ds_sel['size'].max(dim='time')
    surface_max = ds_sel['surface_area'].max(dim='time')
    height_max = ds_sel['max_height'].max(dim='time')
    base_height_min = ds_sel['cloud_base_height'].min(dim='time')
    mass_flux_total = ds_sel['mass_flux'].fillna(0).sum(dim='time')
    merges_total = ds_sel['merges_count'].fillna(0).sum(dim='time')
    splits_total = ds_sel['splits_count'].fillna(0).sum(dim='time')
    max_equiv_radius = ds_sel['max_equiv_radius'].max(dim='time')
    base_radius_diag_max = ds_sel.get('base_radius_diagnosed', xr.zeros_like(size_max)*np.nan).max(dim='time')
    # Fallback: compute prescribed base radius from cloud_base_area if variable absent
    if 'base_radius_prescribed' in ds_sel:
        base_radius_presc_max = ds_sel['base_radius_prescribed'].max(dim='time')
    else:
        base_radius_presc_max = np.sqrt(ds_sel['cloud_base_area'] / np.pi).max(dim='time')
    compact_median = ds_sel['compactness_per_level'].median(dim='level', skipna=True).median(dim='time', skipna=True)

    birth = xr.apply_ufunc(_time_index_first, ds_sel['size'], vectorize=True, input_core_dims=[['time']])
    death = xr.apply_ufunc(_time_index_last, ds_sel['size'], vectorize=True, input_core_dims=[['time']])
    lifetime = death - birth + 1

    df = pd.DataFrame({
        'track_index': ds_sel['track'].values,
        'size_max': size_max.values,
        'surface_area_max': surface_max.values,
        'height_max': height_max.values,
        'base_height_min': base_height_min.values,
        'mass_flux_total': mass_flux_total.values,
        'merges_total': merges_total.values.astype(int),
        'splits_total': splits_total.values.astype(int),
        'max_equiv_radius': max_equiv_radius.values,
        'base_radius_diagnosed_max': base_radius_diag_max.values,
        'base_radius_prescribed_max': base_radius_presc_max.values,
        'compactness_median': compact_median.values,
        'birth_t': birth.values,
        'death_t': death.values,
        'lifetime_steps': lifetime.values
    })
    return df

def pdf_cloud_count(ds, track_sel, ax=None):
    ds_sel = ds.sel(track=track_sel)
    counts = active_mask(ds_sel).sum(dim='track').values
    ax = ax or plt.gca()
    ax.hist(counts, bins='auto')
    ax.set_xlabel('Valid clouds per timestep')
    ax.set_ylabel('Frequency')
    ax.set_title('PDF: number of clouds')

def histogram_merges_splits(df, ax=None):
    ax = ax or plt.gca()
    max_ev = max(df['merges_total'].max(initial=0), df['splits_total'].max(initial=0), 0)
    bins = np.arange(0, max_ev+2) - 0.5
    ax.hist(df['merges_total'], bins=bins, alpha=0.6, label='merges')
    ax.hist(df['splits_total'], bins=bins, alpha=0.6, label='splits')
    ax.set_xlabel('Event count per track'); ax.set_ylabel('Tracks')
    ax.legend()
    ax.set_title('Merge / split distribution')

def pdf_size_and_surface(ds, track_sel, ax_size=None, ax_surface=None):
    ds_sel = ds.sel(track=track_sel)
    sizes = ds_sel['size'].values.ravel()
    surf = ds_sel['surface_area'].values.ravel()
    sizes = sizes[~np.isnan(sizes)]
    surf = surf[~np.isnan(surf)]
    ax_size = ax_size or plt.gca()
    ax_size.hist(sizes, bins=50, log=USE_LOG_SIZE)
    ax_size.set_xlabel('Cloud size (points)'); ax_size.set_ylabel('Count'); ax_size.set_title('Size distribution')
    if ax_surface:
        ax_surface.hist(surf, bins=50)
        ax_surface.set_xlabel('Surface area (m²)'); ax_surface.set_title('Surface area distribution')

def lifetime_distribution(df, ax=None):
    ax = ax or plt.gca()
    ax.hist(df['lifetime_steps'], bins='auto')
    ax.set_xlabel('Lifetime (timesteps)'); ax.set_ylabel('Tracks')
    ax.set_title('Lifetime distribution')

def mass_flux_share(df):
    mf_total = df['mass_flux_total'].sum()
    merged = df.loc[df['merges_total']>0, 'mass_flux_total'].sum()
    frac = merged / mf_total if mf_total>0 else np.nan
    return {'merged_fraction': frac, 'single_fraction': 1 - frac if mf_total>0 else np.nan}

def scatter(df, x, y, ax=None, logx=False, logy=False):
    ax = ax or plt.gca()
    ax.scatter(df[x], df[y], s=14, alpha=0.55)
    ax.set_xlabel(x); ax.set_ylabel(y)
    if logx: ax.set_xscale('log')
    if logy: ax.set_yscale('log')
    ax.grid(alpha=0.3)

def hist2d(df, x, y, ax=None, bins=40):
    ax = ax or plt.gca()
    h = ax.hist2d(df[x], df[y], bins=bins, cmap='viridis')
    plt.colorbar(h[3], ax=ax, label='Counts')
    ax.set_xlabel(x); ax.set_ylabel(y)

def save_fig(fig, name):
    Path(OUT_DIR).mkdir(parents=True, exist_ok=True)
    fig.savefig(Path(OUT_DIR)/name, dpi=140, bbox_inches='tight')

def ratio(a, b):
    r = a / b
    return r[np.isfinite(r)]

print("Utilities loaded.")

Utilities loaded.


In [7]:
# --- Cell 2: Load & Filter ---
ds = open_ds(NETCDF_PATH)
track_selection = filter_tracks(ds)
df = build_track_dataframe(ds, track_selection)
print(f"Total tracks: {ds.dims['track']}")
print(f"Selected valid tracks: {track_selection.sum().item()}")
print("Mass flux share (merged vs single):", mass_flux_share(df))
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/jure/PhD/coding/tracking/cloudtracker/analysis/cloud_results.nc'